```
This notebook sets up and runs a test case for analyzing Kelvin waves
Copyright (C) 2018 - 2022 SINTEF Digital
Copyright (C) 2018 - 2022 Norwegian Meteorological Institute

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

In [ ]:
import sys
gpuocean_path = [p[:-4] for p in sys.path if (p.endswith("gpuocean/src") or p.endswith("gpuocean\\src"))][0]
import git
repo = git.Repo(gpuocean_path)
print("GPUOcean code from:", repo.head.object.hexsha, "on branch", repo.active_branch.name)

# Oslofjord
Testing of Nils projected files

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
from netCDF4 import Dataset
import datetime, copy
from IPython.display import display

#For plotting
import matplotlib
from matplotlib import pyplot as plt

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

In [ ]:
from gpuocean.utils import IPythonMagic, Common, NetCDFInitialization, WindStress, OceanographicUtilities

from gpuocean.SWEsimulators import CDKLM16

In [ ]:
%cuda_context_handler baroclinic_gpu_ctx

Path to the test file

In [ ]:
source_url = "/sintef/data/OsloFjord/test_polstere_1h_0007.nc"

## Inspecting file structure and content

In [ ]:
from netCDF4 import Dataset
nc = Dataset(source_url)

In [ ]:
import xarray as xr
ds = xr.open_dataset(source_url)
ds

Animation utils

In [ ]:
import animation_utils

## Generating GPUOcean Simulation from Input

General parameters

In [ ]:
dimY, dimX = ds.h.data.shape
x0, x1, y0, y1 = 5, dimX-5, 175, dimY-5

Simulation span: 6h!

In [ ]:
t_start = 5*24
t_stop =  t_start + 6

T = (t_stop-t_start)*3600  #Input
timestep_indices = [list(np.arange(t_start, t_stop+1))]

Generating wind fields

In [ ]:
from gpuocean.utils import WindStress

def generate_wind_field(wind_angle_deg, t_start_index, T):
    """Generating WindStress object with spatially constant, hourly varying wind 
    according to the standard shape in Oslofjord (see Oslofjord-ArtificialParameters.ipynb)
    
    wind_angle_deg - angle of wind field towards north in degree!
    t_start_index  - index for the time when generated wind field is supposed to start
    T              - time span for which the wind field is generated in seconds
    """

    wind_angle = np.deg2rad(wind_angle_deg)

    # Reference wind speed for 0 - 24h 
    wind_u_ref = np.zeros(24)
    wind_u_ref[:10] = -0.5*(np.sin(2*np.pi*(np.arange(0,10)-2.5)/10)+1)
    wind_u_ref[10:] = 5*(np.sin(2*np.pi*(np.arange(10,24)+0.5)/14)+1)

    # Wind fields for 0 - 24h
    wind_u = list(np.cos(wind_angle)*wind_u_ref[:,np.newaxis][:,np.newaxis].astype(np.float32))
    wind_v = list(np.sin(wind_angle)*wind_u_ref[:,np.newaxis][:,np.newaxis].astype(np.float32))

    # Shifting that reference hours align with t_0
    shift = datetime.datetime.utcfromtimestamp(nc["ocean_time"][t_start_index]).hour
    for shift_hour in range(shift):
        wind_u.append(wind_u.pop(0))
        wind_v.append(wind_v.pop(0))

    # Repeat for several days if necessary
    wind_u = wind_u * int(np.ceil((T/3600+1)/24))
    wind_v = wind_v * int(np.ceil((T/3600+1)/24))

    # Cut relevant time span
    wind_u = wind_u[0:int(T/3600+1)]
    wind_v = wind_v[0:int(T/3600 +1)]

    # Construct time array in simulator time (starting with 0)
    ts = np.arange(T+1, step=3600)

    # Init WindStress object
    wind = WindStress.WindStress(t=ts, wind_u=wind_v, wind_v=wind_u)
    wind.compute_wind_stress_from_wind()

    return wind 

#### Baroclinic Simulations

In [ ]:
_, ref_baroclinic_data_args = NetCDFInitialization.getCombinedInitialConditions(source_url, x0, x1, y0, y1, 1024.0, timestep_indices=timestep_indices, norkyst_data=False, land_value=0.0, download_data=False)

In [ ]:
ref_baroclinic_data_args["wind"] = generate_wind_field(45.0, t_start, T)
ref_baroclinic_data_args["wind_stress_factor"] = 0.3

In [ ]:
ref_baroclinic_sim = CDKLM16.CDKLM16(baroclinic_gpu_ctx, **NetCDFInitialization.removeMetadata(ref_baroclinic_data_args), dt=0.0, write_netcdf=True)

In [ ]:
bc_eta, bc_hu, bc_hv = ref_baroclinic_sim.download()

fig, axs = plt.subplots(1,3, figsize=(15,5))

im = axs[0].imshow(-bc_eta, vmin=-15, vmax=0, cmap="coolwarm")
axs[0].set_title("MLD")
plt.colorbar(im, ax=axs[0], shrink=0.5)

im = axs[1].imshow(bc_hu, vmin=-5, vmax=5, cmap="coolwarm")
axs[1].set_title("hu")
plt.colorbar(im, ax=axs[1], shrink=0.5)

im = axs[2].imshow(bc_hv, vmin=-5, vmax=5, cmap="coolwarm")
axs[2].set_title("hv")
plt.colorbar(im, ax=axs[2], shrink=0.5)

In [ ]:
subt = 3600
for runt in range(int(T/subt)):
    ref_baroclinic_sim.step(subt)

In [ ]:
# with plt.rc_context({'lines.color':'black', 
#                         'text.color':'black', 
#                         'axes.labelcolor':'black', 
#                         'xtick.color':'black',
#                         'ytick.color':'black'}):
#     anim = animation_utils.ncAnimation(ref_baroclinic_sim.sim_writer.output_file_name, "gpuocean-reduced_grav", comment="GPUOcean")
#     anim.save("anims/baroclinic_test.mp4")

## Baroclinic Drifters

In [ ]:
import pandas as pd 

In [ ]:
baroclinic_wind_angles_degs = np.arange(35, 56, 5)
baroclinic_wind_samples = [None]*len(baroclinic_wind_angles_degs)

for i in range(len(baroclinic_wind_angles_degs)):
    baroclinic_wind_samples[i] = generate_wind_field(baroclinic_wind_angles_degs[i], t_start, T)

In [ ]:
# Mixed layer depth (MLD) 
# Can be explored coupled or decoupled with the reduced gravity constant
mld_dens_samples = np.arange(1022.5, 1024.6, 0.50)
mld_samples_data_args = [None]*len(mld_dens_samples)

for i in range(len(mld_dens_samples)):
    _, mld_samples_data_args[i] = NetCDFInitialization.getCombinedInitialConditions(source_url, x0, x1, y0, y1, 
                                                                                    mld_dens_samples[i], timestep_indices=timestep_indices, norkyst_data=False, land_value=0.0, download_data=False)
    print(mld_samples_data_args[i]["g"])


In [ ]:
wind_stress_samples = np.arange(0.1, 0.51, 0.1)

In [ ]:
friction_samples = np.arange(0, 0.0051, 0.001)

In [ ]:
windage_samples = np.arange(0.0, 0.051, 0.01)

#### Creating simulators

In [ ]:
baroclinic_sims = []
bc_table = pd.DataFrame(columns=["baroclinic_id", "wind_angle_id", "wind_stress_factor_id", "friction_id", "mld_id"]).set_index("baroclinic_id")

for i_w in range(len(baroclinic_wind_samples)):
    for i_ws in range(len(wind_stress_samples)):
        for i_f in range(len(friction_samples)):
            for i_mld in range(len(mld_samples_data_args)):
                baroclinic_data_args = copy.copy(mld_samples_data_args[i_mld])
                baroclinic_data_args["wind"] = baroclinic_wind_samples[i_w]
                baroclinic_data_args["wind_stress_factor"] = wind_stress_samples[i_ws]
                baroclinic_data_args["r"] = friction_samples[i_f]

                baroclinic_sims.append( CDKLM16.CDKLM16(baroclinic_gpu_ctx, **NetCDFInitialization.removeMetadata(baroclinic_data_args),  dt=0.0))
                
                bc_table.loc[len(bc_table.index)] = [i_w, i_ws, i_f, i_mld]

In [ ]:
bc_table

In [ ]:
ref_table = pd.DataFrame(columns=["drifter_id", "baroclinic_id", "windage_id"]).set_index("drifter_id")


for bc in range(len(baroclinic_sims)):
    for windage in range(len(windage_samples)):
        ref_table.loc[len(ref_table.index)] = [bc, windage]

ref_table

Collecting drifter and observation objects

In [ ]:
from gpuocean.utils import Observation
from gpuocean.drifters import GPUDrifterCollection
from gpuocean.dataassimilation import DataAssimilationUtils as dautils

In [ ]:
observation_type = dautils.ObservationType.UnderlyingFlow 
    
observation_args = {'observation_type': observation_type,
                'nx': ref_baroclinic_sim.nx, 'ny': ref_baroclinic_sim.ny,
                'domain_size_x': ref_baroclinic_sim.nx*ref_baroclinic_sim.dx,
                'domain_size_y': ref_baroclinic_sim.ny*ref_baroclinic_sim.dy,
                'land_mask': ref_baroclinic_sim.getLandMask()
                }

trajectories = Observation.Observation(**observation_args)


In [ ]:
crossprod_trajectories = []
for cp in range(len(ref_table)):
    crossprod_trajectories.append(copy.deepcopy(trajectories))

In [ ]:
initx = [ 6000, 8000, 5000,  12000, 15000, 4000]
inity = [12000, 3000, 7500, 22000, 16000, 15000]

num_drifters = len(initx)

In [ ]:
crossprod_drifters = []
for cp in range(len(ref_table)): 
    drifters = GPUDrifterCollection.GPUDrifterCollection(baroclinic_gpu_ctx, # OBS: This is used for wind drift! 
                                                    num_drifters,
                                                    boundaryConditions = ref_baroclinic_sim.boundary_conditions,
                                                    domain_size_x = trajectories.domain_size_x,
                                                    domain_size_y = trajectories.domain_size_y,
                                                    gpu_stream = ref_baroclinic_sim.gpu_stream, # Get re-set in attachment to simulation!
                                                    wind = ref_baroclinic_data_args["wind"],
                                                    wind_drift_factor = windage_samples[ref_table.iloc[cp].windage_id]
                                                    )           

    drifter_pos_init = np.array([initx, inity]).T
    drifters.setDrifterPositions(drifter_pos_init)
    crossprod_drifters.append(drifters)

Attach CPdrifters

In [ ]:
from itertools import compress

In [ ]:
for bc in range(len(baroclinic_sims)):
    baroclinic_sims[bc].attachCrossProductDrifters( 
        list(compress(crossprod_drifters, ref_table["baroclinic_id"] == bc)), 
        [None]*len(windage_samples) )

In [ ]:
for d in range(len(crossprod_drifters)):
    crossprod_trajectories[d].add_observation_from_drifters(crossprod_drifters[d], 0.0)

Run the drift experiment

In [ ]:
for bc in range(len(baroclinic_sims)):
    drifter_ids = ref_table.index[ref_table["baroclinic_id"]==bc].tolist()
    while baroclinic_sims[bc].t < T:
        baroclinic_sims[bc].step(360)
        for d in drifter_ids:
            crossprod_trajectories[d].add_observation_from_drifters(crossprod_drifters[d], baroclinic_sims[bc].t)

In [ ]:
def plot_cp_trajectories(crossprod_trajectories, drifter_id):
    with plt.rc_context({'lines.color':'black', 
                            'text.color':'black', 
                            'axes.labelcolor':'black', 
                            'xtick.color':'black',
                            'ytick.color':'black'}):
        fig, ax = plt.subplots(1,1, figsize=(10,10))
        ax.tick_params(axis='both', which='major', labelsize=28)
        domain_extent = [0, ref_baroclinic_sim.nx*ref_baroclinic_sim.dx/1000, 0, ref_baroclinic_sim.ny*ref_baroclinic_sim.dy/1000]

        bg_cmap = copy.deepcopy(plt.cm.Blues)
        bg_cmap.set_bad("grey", alpha = 1.0)

        ax.imshow(ref_baroclinic_sim.download()[0] > -10, interpolation="none", origin='lower', 
                        cmap=bg_cmap,  
                        extent=domain_extent)

        path = crossprod_trajectories[0].get_drifter_path(drifter_id, 0, T, in_km = True)[0]
        start_pos = path[0,:]
        circ_start = matplotlib.patches.Circle((start_pos[0], start_pos[1]), 
                                                0.01, color = 'black', fill=True, zorder=10)
        ax.add_patch(circ_start)

        for path in [t.get_drifter_path(drifter_id, 0, T, in_km = True)[0] for t in crossprod_trajectories]:
            
            ax.plot(path[:,0], path[:,1], color="C0", ls="-", zorder=5, alpha=0.1)
            
            end_pos = path[-1,:]
            ax.scatter(end_pos[0], end_pos[1], marker='x', color='black', s=100, linewidths=2)

        plt.tight_layout()

        plt.savefig("figs/CPdrift"+str(drifter_id)+".pdf", bbox_inches="tight")

In [ ]:
for drifter_id in range(num_drifters):
    plot_cp_trajectories(crossprod_trajectories, drifter_id)

In [ ]:
os.makedirs("pickles", exist_ok=True)
for cp in range(len(crossprod_trajectories)):
    crossprod_trajectories[cp].to_pickle("pickles/cp_trajectory"+str(cp))


In [ ]:
STOP

In [ ]:
from gpuocean.utils import Observation

In [ ]:
obs= Observation.Observation()
obs.read_pickle('pickles/2023-03-17T16_26_35/cp_trajectory0')

In [ ]:
trajectories = []
for f in os.listdir('pickles/2023-03-17T16_26_35'):
    obs= Observation.Observation()
    obs.read_pickle('pickles/2023-03-17T16_26_35/'+f)
    trajectories.append(obs )

In [ ]:
plot_cp_trajectories(trajectories, 0)